# Project Code 

In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.impute import KNNImputer


## Cleaning

In [2]:
# done outside of the datacheckpoint: 
# impute missing values using knnimputer using k = 5 clusters to fill in missing GPA received
# fill in missing letter grades using .apply 
#capes = pd.read_csv('CAPES.csv')
## YOUR CODE TO LOAD/CLEAN/TIDY/WRANGLE THE DATA GOES HERE
capes = pd.read_csv("capes_data.csv")
## define some functions to help clean the data

# takes in the full code for an individual class and gets just the class code without the title
def extract_class_code(full_name):
    return full_name.split()[0] + full_name.split()[1]

# takes in the full code for an individual class and gets the subject code
def extract_subject_code(full_name):
    return full_name.split()[0]

# CAPES formats its grades as 'lettergrade (GPA)' i.e. 'A+ (4.00)'
# these functions take in the CAPES formatted grades and extracts the GPA and letter grade respectively
def extract_GPA(grade):
    if type(grade) == str:
        return float(grade.split()[1][1:-1])
    return np.NaN

def extract_letter_grade(grade):
    if type(grade) == str:
        return grade.split()[0]
    return np.NaN

# apply functions to the columns
class_codes = capes['Course'].apply(extract_class_code)
subject_codes = capes['Course'].apply(extract_subject_code)
gpa_expected = capes['Average Grade Expected'].apply(extract_GPA)
gpa_received = capes['Average Grade Received'].apply(extract_GPA)
letter_expected = capes['Average Grade Expected'].apply(extract_letter_grade)
letter_received = capes['Average Grade Received'].apply(extract_letter_grade)

# assign to CAPES dataframe
capes = capes.assign(**{'Average Grade Expected' : letter_expected})
capes = capes.assign(**{'Average Grade Received' : letter_received})
capes = capes.assign(**{'Average GPA Expected' : gpa_expected})
capes = capes.assign(**{'Average GPA Received' : gpa_received})
capes = capes.assign(**{'Class Code' : class_codes})
capes = capes.assign(**{'Subject Code' : subject_codes})
# drop columns that are not variables we want to study
capes = capes.drop(['Instructor', 'Percentage Recommended Class', 'Percentage Recommended Professor', 'Evaluation URL'], axis = 1)

# missing values
print(capes.isna().any())
# impute using KNNImputer
impute = KNNImputer()
capes[['Study Hours per Week', 'Average GPA Expected', 'Average GPA Received']] = impute.fit_transform(capes[['Study Hours per Week', 'Average GPA Expected', 'Average GPA Received']])

print(capes)

# round the results from KNN Imputer
capes['Average GPA Expected'] = np.round(capes['Average GPA Expected'], 2)
capes['Average GPA Received'] = np.round(capes['Average GPA Received'], 2)

# function to convert GPA to letter grade
def to_letter(GPA):
    if GPA == 4:
        return 'A'
    if GPA >= 3.7:
        return 'A-'
    if GPA >= 3.3:
        return 'B+'
    if GPA >= 3.0:
        return 'B'
    if GPA >= 2.7:
        return 'B-'
    if GPA >= 2.3:
        return 'C+'
    if GPA >= 2.0:
        return 'C'
    if GPA >= 1.7:
        return 'C-'
    if GPA >= 1:
        return 'D'
    return 'F'

# convert to letter grade to fill in missing values
capes['Average Grade Expected'] = capes['Average GPA Expected'].apply(to_letter)
capes['Average Grade Received'] = capes['Average GPA Received'].apply(to_letter)

print(capes.isna().any())

capes.to_csv("CAPES.csv")
capes

Course                      False
Quarter                     False
Total Enrolled in Course    False
Total CAPEs Given           False
Study Hours per Week         True
Average Grade Expected       True
Average Grade Received       True
Average GPA Expected         True
Average GPA Received         True
Class Code                  False
Subject Code                False
dtype: bool
                                               Course Quarter  \
0         AAS 10 - Intro/African-American Studies (A)    SP23   
1                  AAS 170 - Legacies of Research (A)    SP23   
2           ANAR 111 - Foundations of Archaeology (A)    SP23   
3        ANAR 115 - Coastal Geomorphology/Environ (A)    SP23   
4       ANAR 155 - Stdy Abrd: Ancient Mesoamerica (A)    SP23   
...                                               ...     ...   
63358  VIS 105D  - Aesthetics/Chinese Calligraphy (A)    SU07   
63359  VIS 106A  - Painting:  Image Making        (A)    SU07   
63360  VIS 194S  - Fantasy In

,Course,Quarter,Total Enrolled in Course,Total CAPEs Given,Study Hours per Week,Average Grade Expected,Average Grade Received,Average GPA Expected,Average GPA Received,Class Code,Subject Code
0,AAS 10 - Intro/African-American Studies (A),SP23,66,48,2.80,A-,B+,3.84,3.67,AAS10,AAS
1,AAS 170 - Legacies of Research (A),SP23,20,7,2.50,A-,A-,3.86,3.92,AAS170,AAS
2,ANAR 111 - Foundations of Archaeology (A),SP23,16,3,3.83,B+,B+,3.67,3.53,ANAR111,ANAR
3,ANAR 115 - Coastal Geomorphology/Environ (A),SP23,26,6,3.83,B+,B,3.50,3.07,ANAR115,ANAR
4,ANAR 155 - Stdy Abrd: Ancient Mesoamerica (A),SP23,22,9,5.17,A,A,4.00,4.00,ANAR155,ANAR
...,...,...,...,...,...,...,...,...,...,...,...
63358,VIS 105D - Aesthetics/Chinese Calligraphy (A),SU07,17,13,2.33,A,A-,4.00,3.93,VIS105D,VIS
63359,VIS 106A - Painting: Image Making (A),SU07,16,14,7.21,A-,B+,3.79,3.54,VIS106A,VIS
63360,VIS 194S - Fantasy In Film (A),SU07,80,57,4.27,B,B,3.21,3.20,VIS194S,VIS
63361,VIS 22 - Formations of Modern Art (A),SU07,40,33,4.32,B+,B+,3.62,3.47,VIS22,VIS


## Percentiles

To filter down our dataset, we decided to use percentiles to determine which classes to analyze. Some departments are mostly made of classes that have high weights, meaning most of the students of that department will take these classes. Other departments have majors with several electives that are not required, but are an option some majors. We decided to take the 50th percentile to get a good sense of the median weight for the department. Any class with a weight greater than or equal to the median would be included. 

In [3]:
combined_weights = pd.read_csv('Combined Weights.csv')

In [4]:
combined_weights
for dept in combined_weights['Department'].unique():
    df = combined_weights[combined_weights['Department'] == dept].sort_values('Combined Weights', ascending = False)
    #print(np.percentile(df['Combined Weights'], q = 25))
    #print(np.percentile(df['Combined Weights'], q = 50))
    #print(np.percentile(df['Combined Weights'], q = 75))
    #display(df)

major_count = combined_weights.groupby(['Department'])['Major'].aggregate(lambda x : x.nunique())
percentiles = combined_weights.groupby('Department')['Combined Weights'].aggregate([lambda x : np.percentile(x, 25) , lambda x : np.percentile(x, 50), lambda x : np.percentile(x, 75)])
perc = percentiles.rename(columns={'<lambda_0>': "25th Percentile", '<lambda_1>' : "50th Percentile", '<lambda_2>': "75th Percentile"}) 
for i in perc.columns:
    perc[i] = perc[i] / major_count

def above_50(x):
    #print((x['Combined Weights'] >= np.percentile(x['Combined Weights'], 50)))
    return x[(x['Combined Weights'] >= np.percentile(x['Combined Weights'], 50))]
filtered = combined_weights.groupby('Department').apply(above_50).drop(columns = 'Department').reset_index().drop(columns = 'level_1')
filtered

,Department,Class,Major,Weight,Combined Weights
0,BIO,BILD1,BI34,1.0,7.000000
1,BIO,BILD3,BI34,1.0,7.000000
2,BIO,BILD4,BI34,1.0,7.000000
3,BIO,CHEM6A,BI34,1.0,7.000000
4,BIO,CHEM6B,BI34,1.0,7.000000
...,...,...,...,...,...
1620,PSYCH,PSYC102,PC35,0.4,4.792857
1621,PSYCH,PSYC104,PC35,1.0,6.642857
1622,PSYCH,PSYC105,PC35,1.0,5.392857
1623,PSYCH,PSYC106,PC35,0.4,4.792857


Since the median is being used as a threshold value, not every department is has exactly 50% of the original classes that were originally collected. Two thirds of the International classes, for example, have a weight of 12. This means that every International major is required to take these classes. 

In [5]:
percent_of_original = filtered.groupby('Department').count()['Class'] / combined_weights.groupby('Department').count()['Class']
percent_of_original
#filtered.to_csv('filtered_weights.csv') 

Department
BIO        0.518519
CHEM       0.529412
COG SCI    0.518957
CS         0.511111
ECE        0.548023
ECON       0.513889
INTL       0.666667
MAE        0.540541
MATH       0.519608
PSYCH      0.558673
Name: Class, dtype: float64